<H1 align="center">Proyecto Final - IntelliPrice</H1>
<h3>Grupo: <i>Quantum Coders</i> - Chatbot Comparador de Productos</h3>
Integrantes del grupo: Nikechi Camarena &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Aula: P02 - Panamá <br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Lucas Castillo<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Gabriel Cortez<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Sergio López<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;María Suira


<H3>1. Se importan las librerias a utilizar.</h3>

In [19]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import tkinter as tk
from tkinter import *
import json
import pandas as pd
import re
import unicodedata

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slope\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\slope\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<H3>2. Se carga el dataset de precios y productos para las comparaciones.</h3>

In [20]:

# Unir los DataFrames en base al 'producto_id'
df_productos = pd.read_csv("Lista_productos.csv", delimiter=";")
df_productos

,precio,sucursal,id,marca,nombre,presentacion
0,337.00,Sol y Mar,7798123131611,GREAT VALUE,Liquido Lavavajillas,500.0 cc
1,337.00,La Hacienda,7798123131611,GREAT VALUE,Liquido Lavavajillas,500.0 cc
2,279.00,El Refugio,7791172001348,DIAMANTE,Liquido Lavavajillas,500.0 ml
3,309.00,Las Margaritas,7791172001348,DIAMANTE,Liquido Lavavajillas,500.0 ml
4,319.99,El Retiro,8000580204906,FINISH,Liquido Lavavajillas,500.0 ml
...,...,...,...,...,...,...
320,38.50,Plaza del Mar,7791337001299,DANONINO,Yogurt,80.5 gr
321,43.40,Calle de las Flores,7791337001299,DANONINO,Yogurt,80.5 gr
322,42.40,El Rincón,7791337001299,DANONINO,Yogurt,80.5 gr
323,40.00,Gran Plaza,7791337001299,DANONINO,Yogurt,80.5 gr


<H3>3. Se extraen los productos y sucursales para tener una lista con cada producto disponible en el Chatbot.</h3>

In [21]:
product_names = df_productos['nombre'].unique()
print(product_names.size)
product_names

17


array(['Liquido Lavavajillas', 'Aceite para bebe', 'Aceite de Canola',
       'Café Instantaneo', 'Crema Dental', 'Crema Facial',
       'Desinfectante', 'Enjuage Bucal', 'Escoba', 'Jabon de Baño',
       'Lentejas', 'Mayonesa', 'Pan', 'Pañal', 'Shampoo', 'Vino Blanco',
       'Yogurt'], dtype=object)

In [22]:
branch_name = df_productos['sucursal'].unique()
print(branch_name.size)
branch_name

50


array(['Sol y Mar', 'La Hacienda', 'El Refugio', 'Las Margaritas',
       'El Retiro', 'La Sierra', 'El Manantial', 'El Pueblo',
       'Avenida del Sol', 'Plaza del Mar', 'Calle de las Flores',
       'Paseo de los Sueños', 'Boulevard de la Luna', 'El Rincón',
       'Gran Plaza', 'Mercado del Sol', 'El Álamo', 'Gran Sur',
       'La Estrella', 'La Fuente', 'Mercado Real', 'El Faro',
       'Costa Azul', 'El Encanto', 'Brisa Marina', 'Las Colinas',
       'Las Rocas', 'El Bosque', 'La Reserva', 'Monte Verde',
       'El Horizonte', 'Primavera', 'La Casona', 'Los Laureles',
       'El Paraíso', 'La Campiña', 'Vía Verde', 'Buenavista', 'La Senda',
       'La Cumbre', 'El Triunfo', 'La Floresta', 'La Esperanza',
       'El Campestre', 'Bella Vista', 'La Laguna', 'Campo Alegre',
       'Las Fuentes', 'Valle Dorado', 'La Aurora'], dtype=object)

<H3>4. Se cargan los datos de entrenamientos del intents.json y se hace el procesado de estos.</h3>

In [23]:
lemmatizer = WordNetLemmatizer()

# Se cargan los Intents
with open('intents.json') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']


# Procesamiento de Intents para el entrenamiento del modelo
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

train_x = []
train_y = []

for i in training:
    train_x.append(i[0])
    train_y.append(i[1])

train_x = np.array(train_x)
train_y = np.array(train_y)

# Función para limpiar y lematizar oración
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Función para convertir oración a bolsa de palabras
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))


<H3>5. Se crea la clase que predecirá los inputs del usuario.</h3>

In [24]:
# Función para predecir la clase con el modelo que se creará
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    return return_list

<H3>6. Se establecen las funciones para responder al usuario, haciendo busquedas en el dataframe creado.</h3>

In [25]:
# Función para obtener respuesta
def get_response(ints, intents_json, msg):
    
    tag = ints[0]['intent']
    print(tag)
    list_of_intents = intents_json['intents']

    if tag == 'products_list':
        response = "Estos son los productos disponibles:\n" + "\n".join(product_names)
        return response
    elif tag == 'product_query':
        product_name = get_product_name(msg)
        prices = None
        if product_name:
            prices = get_product_prices(product_name)
        if prices:
            response = f"Estos son los 10 precios más bajos que encontré para {product_name}:\n"
            sorted_prices = sorted(prices, key=lambda x: x['precio'])[:10]
            for price in sorted_prices:
                response += "- Sucursal {}: ${:.2f} - {}\n".format(price['sucursal'], price['precio'], price['presentacion'])
            return response
        else:
            return "Lo siento, no encontré ese producto. Podrías preguntar por la lista de productos disponibles."
    elif tag == 'sucursales_disponibles':
        response = "Estas son las sucursales disponibles: " + "\n".join(branch_name)
        return response
    elif tag == 'productos_sucursal':
        branch = get_branch_name(msg)
        if branch:
            product_branch = df_productos[df_productos['sucursal'] == branch]
            response = f"Los productos disponibles en la sucursal {branch} son:\n"
            print(response)
            for index, item in product_branch.iterrows():
                response += "- {} {} de {}\n".format(item['nombre'], item['marca'], item['presentacion'])
            print(response)
            return response
        else:
            return "Esa sucursal no se encuentra en nuestra base de datos."

    else:
        for intent in intents['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']
        return random.choice(responses)    

def get_product_prices(product_name):
    result = df_productos[df_productos['nombre'].str.contains(product_name, case=False, na=False)]
    if not result.empty:
        return result[['sucursal', 'precio', 'marca', 'nombre', 'presentacion']].to_dict(orient='records')
    else:
        return None


def normalizar_texto(texto):
    # Eliminar acentos y caracteres especiales
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')

    # Eliminar signos de puntuación y caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    return texto

#Se implementan busquedas utilizando Operaciones de conjuntos

def get_branch_name(frase):
    frase_normalizada = normalizar_texto(frase.lower())
    palabras_usuario = set(frase_normalizada.split())

    for branch in branch_name:
        b_normalizado = normalizar_texto(branch.lower())
        palabras_branch = set(b_normalizado.split())
        if palabras_branch.issubset(palabras_usuario):
            return branch
    return None



def get_product_name(frase):
    frase_normalizada = normalizar_texto(frase.lower())
    
    palabras_usuario = set(frase_normalizada.split())
    
    for p in product_names:
        p_normalizado = normalizar_texto(p.lower())
        palabras_producto = set(p_normalizado.split())
        if palabras_producto.issubset(palabras_usuario):
            return p

    return None

<H3>7. Se crea una clase que contendrá el chatbot, su creación, entrenamiento y funcionamiento.</h3>

In [26]:
class Chatbot:
    def __init__(self):
        self.model = self.create_model()
        self.train_model()
        self.intents = intents

    def create_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))
        sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        loss, accuracy = model.evaluate(train_x, train_y)
        print(accuracy)
        return model

    def train_model(self):
        try:
            self.model.load_weights("model.weights.h5")
        except:
            self.model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
            self.model.save_weights("model.weights.h5")


    def chat(self, sentence):
        ints = predict_class(sentence, self.model)
        res = get_response(ints, self.intents, sentence)
        return res


chatbot = Chatbot()


c:\Users\slope\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\slope\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0498 - loss: 2.4980  
0.06060606241226196


c:\Users\slope\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<H3>8. Se crea la interfaz gráfica del Chatbot.</h3>

In [27]:
class ChatGUI:
    
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("IntelliPrice Chatbot")

        # Mejorar nitidez del texto en Windows
        try:
            from ctypes import windll
            windll.shcore.SetProcessDpiAwareness(1)
        except:
            pass

        # Configurar colores y fuentes
        self.app_bg_color = "#e0e0e0"  # Color de fondo de la aplicación (gris claro)
        self.text_bg_color = "#f0f0f0"  # Color de fondo del cuadro de texto (gris un poco más oscuro)
        self.text_color = "#333333"  # Color de texto
        self.entry_bg_color = "#ffffff"  # Color de fondo de la entrada
        self.button_bg_color = "#1428a0"  # Color de fondo del botón
        self.button_text_color = "#ffffff"  # Color de texto del botón
        self.title_bg_color = "#1428a0"  # Color de fondo del título
        self.title_text_color = "#ffffff"  # Color de texto del título
        self.font = ("Arial", 16)
        self.entry_font = ("Arial", 14)
        self.button_font = ("Arial", 14, "bold")
        self.title_font = ("Arial", 26, "bold")
        

        self.window.config(bg=self.app_bg_color)

        # Añadir título
        self.title_frame = Frame(self.window, bg=self.title_bg_color)
        self.title_frame.pack(fill=tk.X)
        self.title_label = tk.Label(self.title_frame, text="Bienvenido a IntelliPrice", bg=self.title_bg_color, fg=self.title_text_color, font=self.title_font, pady=10)
        self.title_label.pack(pady=10)

        self.cta_label = tk.Label(self.window, text="¡Descubre dónde ahorrar más en tus compras!", bg=self.app_bg_color, fg="#000000", font=("Arial", 16))
        self.cta_label.pack(pady=10)

        self.chat_frame = Frame(self.window, bg=self.text_bg_color)
        self.chat_frame.pack(padx=10, pady=10)

        self.text_widget = Text(self.chat_frame, width=60, height=20, bg=self.text_bg_color, fg=self.text_color, font=self.font, wrap="word", relief="flat", bd=0)
        self.text_widget.pack(side=LEFT, padx=10, pady=10)
        self.text_widget.config(state=DISABLED)

        self.scrollbar = Scrollbar(self.chat_frame)
        self.scrollbar.pack(side=RIGHT, fill=Y)
        self.text_widget.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_widget.yview)

        self.entry = Entry(self.window, width=55, bg=self.entry_bg_color, fg=self.text_color, font=self.entry_font, insertbackground=self.text_color, relief="flat", bd=1)
        self.entry.pack(padx=10, pady=10)
        self.entry.bind("<Return>", self.send_message)

        self.send_button = Button(self.window, text="Enviar", command=self.send_message, bg=self.button_bg_color, fg=self.button_text_color, font=self.button_font, bd=0, padx=10, pady=5)
        self.send_button.pack(pady=10)


        
    def send_message(self, event=None):
        message = self.entry.get()
        self.entry.delete(0, END)
        
        if message:
            self.display_user_message(f"{message}\n")
            response = chatbot.chat(message) 
            self.display_bot_message(f"{response}\n")

    def display_user_message(self, message):
        self.text_widget.tag_configure("black", foreground="#000000", font=("Arial", 16, "bold"))
        self.text_widget.config(state=NORMAL)
        self.text_widget.insert(END, "Tu: ", "black")
        self.text_widget.insert(END, message)
        self.text_widget.config(state=DISABLED)
        self.text_widget.yview(END)

    def display_bot_message(self, message):
        self.text_widget.tag_configure("blue", foreground="#1428a0", font=("Arial", 16, "bold"))
        self.text_widget.config(state=NORMAL)
        self.text_widget.insert(END, "Bot: ", "blue")
        self.text_widget.insert(END, message)
        self.text_widget.config(state=DISABLED)
        self.text_widget.yview(END)

    def run(self):
        self.window.mainloop()

chat_gui = ChatGUI()
chat_gui.run()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
[{'intent': 'greeting', 'probability': '0.9992493'}]
greeting
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[{'intent': 'product_query', 'probability': '0.92388886'}]
product_query
